# CRISP - DM Method

+ Business understanding – What does the business need?
+ Data understanding – What data do we have / need? Is it clean?
+ Data preparation – How do we organize the data for modeling?
+ Modeling – What modeling techniques should we apply?
+ Evaluation – Which model best meets the business objectives?
+ Deployment – How do stakeholders access the results?

# 1. Business understanding

#### Geschäftsführung bzw. Disposition des Unternehmens 

- Darstellung allgemeiner Erkenntnisse aus den Daten <br>
&rarr; Um E-Scooter-Geschäftsfeld besser zu durchdringen und um Rückschlüsse für die eigenen betrieblichen Handlungsbereiche aus den Daten zu ziehen



#### Abteilungsleiter

- Analysen sollen somit das operative Geschäft des Dispositionsleiters unterstützen <br>
&rarr; Wiederkehrende „Muster“ in den Daten sind dabei von besonderem Interesse

# 2. Data Understanding

In [42]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np

In [ ]:
df = pd.read_parquet("escooter_history_2022.parquet")

# 3. Data Preperation

In [ ]:
# Temperaturspitzen entfernen
df = df[df.temp <75]

# Rechtschreibfehler korrigiert
df.replace({"weather":{"heacy rain or thunderstorm or snow or ice pallets":"heavy rain or thunderstorm or snow or ice pallets"}}, inplace=True)

# Wochentag einfügen
df["weekday"] = df["datetime"].dt.day_name()

# Umwandeln der verschiedenen Bool-Werte in 1 0 für späteres ML-Modell
df.registered_customer = df.registered_customer.astype(int)
df.holiday = df.holiday.astype(int)
df.workingday = df.workingday.astype(int)

Spalte mit daytype generieren

In [ ]:
# Herausfiltern der Übergangszeiten in denen falsche Einträge bei holiday und workingday sind
df_weekend = df[(df.datetime.dt.hour>1)&(df.datetime.dt.hour <24)]

# Basierend auf den Spaltenwerten von holiday und workingday Tagestypen auswählen und in neue Spalte daytype schreiben
conditions= [df_weekend.holiday ==1, df_weekend.workingday==1, df_weekend.holiday == df_weekend.workingday]
choices = ["holiday","workingday", "weekend"]
df_weekend["daytype"] = np.select(conditions, choices)
df_weekend["date"] = df_weekend.datetime.dt.date
# neue Spalte date im Hauptdataframe erzeugen
df["date"] = df.datetime.dt.date
df_weekend = df_weekend.groupby([df_weekend.date, df_weekend.daytype], as_index=False)["datetime"].count().drop(columns="datetime")
# An allen Datensätzen an Date datetype mergen
df = df.merge(df_weekend, left_on="date", right_on="date")

In [ ]:
# Hinzufügen weiterer Spalten zur einfacheren Gruppierung
df["day"] = df.datetime.dt.day
df["hour"] = df.datetime.dt.hour
df['ym-date'] = df.datetime.dt.strftime('%Y-%m')

# 4. Modeling

# 5. Evaluation